Automatically Scoring Student Responses (Embedded responses)

Avi Dixit and Elizabeth McBride

<b>Introduction </b> Notebook to upload the embedded responses data into pandas dataframes and apply classification algorithms to it.


In [76]:
#imports - Consolidated imports for all functions used (or will eventually be used) by the notebook

import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
%matplotlib inline

import re
import math
import string
from collections import Counter
from __future__ import division
from time import time
import logging

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [77]:
# Code to create plot confusion matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

class_names = [1, 2, 3, 4, 5, 6, 7, 8]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [78]:
# Code to read in a file
def read_file(filename):
    #get the file
    df = pd.read_excel(filename)
    
    #Force KIScore to int, otherwise reverts to float. Same for Answer. Forcing NaN to unicode
    df['QuestionType'] = df['QuestionType'].astype(int)
    #df = df[df['QuestionType'] == 1]
    df['Score'] = df['new_Score3'].astype(int)
    df['Text'] = df['Text'].astype(str)

    return df

#reads in the training data into a panda 
#(code based on ANLP Notebook Intro to Pandas by Marti Hearst and Andrea Gagliano)
def read_training_data(filename):
    df_narrative = read_file(filename)
    return df_narrative

### Seperate the data into training and dev data. ###

In [79]:
#breaks the panda into a training set and a dev set 

def get_train_and_dev_sets(full_data, percent_dev):
    # randomize the indices
    random_index = np.random.permutation(full_data.index)
    full_data_shuffled = full_data.ix[random_index, ['Text', 'Score']]
    full_data_shuffled.reset_index(drop=True, inplace=True)
    
    # break down the counts for the shuffled data
    rows, columns = full_data_shuffled.shape
    train_size = round(rows*(1 - percent_dev))
    dev_size   = round(rows*percent_dev)
    
    # separate the training data from the development data
    train_data = full_data_shuffled.loc[:train_size]
    dev_data = full_data_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)

    return train_data, dev_data 

In [80]:
# reads in the test file into a panda
def read_test_data(filename):
    # get the file
    df = read_file(filename)
    return df

In [106]:
#the code that calls the above functions - puts the data into a data frame
df = read_training_data("EmbeddedOvens2.xlsx")
train_set, dev_set = get_train_and_dev_sets(df,.3)

### Spell checker created by Peter Norvig ###
*Not used in final algorithm

In [107]:
TEXT = open('big.txt').read()

def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    #print(re.findall('[a-z]+', text.lower()))
    return re.findall('[a-z]+', text.lower())

def tokens_target(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    words = re.findall('[a-z]+', text.lower())
    tagged_POS_sents = nltk.pos_tag(words) # tags sents
    #normed_tagged_words = [wnl.lemmatize(word[0].lower()) for sent in tagged_POS_sents
                           #for word in sent 
                           #if word[0].lower() not in nltk.corpus.stopwords.words('english')
                           #and word[0] not in punctuation # remove punctuation
                           #and not re.search(r'''^[\.,;"'?!():\-_`]+$''', word[0])
                           #and word[1].startswith('N')]  # include only nouns
    #print(tagged_POS_sents)
    if (len(tagged_POS_sents) > 1):
        normed_tagged_words = [word[0].lower() for word in tagged_POS_sents
                              if (word[1].startswith('N') or word[1].startswith('J') or word[1].startswith('V'))]
        return normed_tagged_words
    else:
        return words

WORDS = tokens(TEXT)

COUNTS = Counter(WORDS)

In [108]:
def correct(word):
    "Find the best spelling correction for this word."
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

# Show what happens in the case of ties
def correct_under_hood (word):
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return candidates

In [109]:
def known(words):
    "Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}

def edits2(word):
    "Return all strings that are two edits away from this word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [110]:
def edits1(word):
    "Return all strings that are one edit away from this word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [111]:
spell_checker = lambda x : ' '.join(i for i in list(map(correct, tokens(x))))

In [112]:
# Normalizes text to be all lowercase
normalizer = lambda x : ' '.join(i for i in list(map(correct, tokens_target(x))))

### Normalize and/or Spellcheck Training and Dev Sets ###

In [113]:
#train_set['Text'] = train_set['Text'].apply(spell_checker)
train_set['Text'] = train_set['Text'].apply(normalizer)

In [114]:
train_set.head()

,Text,Score
0,oven works putting plexiglas top box putting t...,1
1,final solar worked did add more reflection mat...,2
2,nan,1
3,nan,1
4,are first solar work champ second one was dog ...,1


In [115]:
#dev_set['Text'] = dev_set['Text'].apply(spell_checker)
dev_set['Text'] = dev_set['Text'].apply(normalizer)

In [116]:
dev_set.head()

,Text,Score
0,solar oven was using radiation light rays hitt...,1
1,oven be wide short have aluminium bottom top c...,2
2,heat project black paper foil melt food,1
3,heat bones tin foil box,2
4,behave tin foilflap reflect more light box pla...,1


### Prepare data to be classified ###

In [117]:
# Transforms data frames to arrays 
def transform_dfs_to_arrays(train_set, dev_set):
    vec = CountVectorizer(ngram_range=(1, 4), token_pattern=r'\b\w+\b', stop_words="english", max_features=5000)
    arr_train_feature_sparse = vec.fit_transform(train_set["Text"].values.astype(str))
    arr_train_feature = arr_train_feature_sparse.toarray()
    
    arr_dev_feature_sparse = vec.transform(dev_set["Text"].values.astype(str))
    arr_dev_feature = arr_dev_feature_sparse.toarray()
        
    return arr_train_feature, arr_dev_feature


In [118]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        #self.snow = SnowballStemmer('english')
    
    # this code will filter punctuation from a word and rejoin it together ("they're" becomes "theyre")
    def __preprocess(self, doc):
       filter_punc = lambda t: ''.join([x.lower() for x in t if x.isalpha()])
       words = [x for x in map(filter_punc, doc.split()) if x]
       review = ""
       for w in words:
           review = review+" "+w
       return review
    
    #Multiple attempts to select lemmas and stems from a word token (using NLTK)
    def __call__(self, doc):
        #return [self.wnl.lemmatize(t.lower()) for t in word_tokenize(doc)]
        return [self.wnl.lemmatize(t) for t in word_tokenize(self.__preprocess(doc))]
        #return ["".join([str(s.name()) for s in wn.synset(t).hypernyms()]) for t in word_tokenize(self.__preprocess(doc))]
        #return [self.snow.stem(t) for t in word_tokenize(self.__preprocess(doc))]

In [119]:
from collections import Counter
from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from string import punctuation
    
def stuff(doc):
    #flatten = [w for sent in doc for w in sent]
    flatten = [w for w in word_tokenize(doc)]
    unigram_counts = Counter(flatten)
    uni_dist = FreqDist(unigram_counts)
    uni = [a for (a, b) in uni_dist.most_common(25)]
    
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(doc) # Split text into sentences
    words = [nltk.word_tokenize(word) for word in raw_sents]
    wnl = WordNetLemmatizer() # to get word stems
    tagged_POS_sents = [nltk.pos_tag(word) for word in words ] # tags sents
    #print(tagged_POS_sents)
    #normed_tagged_words = [wnl.lemmatize(word[0].lower()) for sent in tagged_POS_sents
                           #for word in sent 
                           #if word[0].lower() not in nltk.corpus.stopwords.words('english')
                           #and word[0] not in punctuation # remove punctuation
                           #and not re.search(r'''^[\.,;"'?!():\-_`]+$''', word[0])
                           #and word[1].startswith('N')]  # include only nouns
    normed_tagged_words = [word[0].lower() for sent in tagged_POS_sents
                          for word in sent
                          if (word[1].startswith('N') or word[1].startswith('J'))]
    #normed_tagged_words = list(set(normed_tagged_words))
    return normed_tagged_words

#from http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
class LemmaTokenizer1(object):
    def __init__(self):
        self.wnl = PorterStemmer()
    def __call__(self, doc):
        return [t for t in stuff(doc)]

### Custom Classifier ###

In [124]:
# Custom Classifier
#Starting with a feature on the length of the answer
#Every class created for the custom feature needs to have a method to transform and fit the data
# Weights - 3 3 1 2 1

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from nltk.stem import PorterStemmer
from sklearn.ensemble import ExtraTreesClassifier
ps = PorterStemmer()


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text)}
                for text in posts.tolist()]

class Keywords_Attract(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'Attract': 'attract' in [ps.stem(i) for i in text.split()]
                or 'attrakt' in [ps.stem(i) for i in text.split()]
                or 'atract' in [ps.stem(i) for i in text.split()]
                or 'atrakt' in [ps.stem(i) for i in text.split()]
                or 'attrackts' in [ps.stem(i) for i in text.split()]
                or 'attracts' in [ps.stem(i) for i in text.split()]}
                for text in posts.tolist()]
    
#Required to convert a sparse matrix to a dense matrix. Vectorizers give out a sparse matrix but some 
#classifiers need a dense matrix to perform classification
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self
    
from sklearn.pipeline import Pipeline, FeatureUnion


clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1, n_estimators = 100)
clf3 = GaussianNB()
clf4 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                      max_depth=1, random_state=0)
clf5 = SGDClassifier(loss='hinge', alpha=1e-5, penalty='elasticnet', n_iter=50, random_state=69)
clf6 = SVC(C = 1000000.0, gamma='auto', kernel='rbf', probability = True)
clf8 = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

eclf_w = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), 
                                    ('clf6', clf6),
                                   ('clf4', clf4), ('clf8', clf8)], 
                                                  weights=[3, 3, 1, 2, 1], voting = 'soft')

pipeline1 = Pipeline([
    ('features', FeatureUnion(
        transformer_list=[
        ('body_stats', Pipeline([
                    ('stats', TextStats()),  # returns a list of dicts
                    ('vect', DictVectorizer())  # list of dicts -> feature matrix
        ])),
        ('key_words_attract', Pipeline([ # Give low weight
                    ('Attract', Keywords_Attract()),  # returns a list of dicts
                    ('vect', DictVectorizer())  # list of dicts -> feature matrix
        ])),
        
        ('bag_of', Pipeline([
                    ('vect', CountVectorizer(ngram_range=(1, 3), tokenizer=LemmaTokenizer(),  
                                              max_df=0.25, max_features= 15000, token_pattern=r'\b\w+\b', 
                                              stop_words="english"))
                    #('tfidf_transformer', TfidfTransformer(use_idf = True, norm='l2'))
        ]))
    ],
    # weight components in FeatureUnion
        #transformer_weights={
            #'body_stats': 1,
            #'key_words_radiate': 1.0,
            #'key_words_trap': 1.0,
            #'bag_of': 1.0        
        #},
    )),
    ('to_dense', DenseTransformer()),   
    ('feature_selection', SelectFromModel(ExtraTreesClassifier(), prefit=False)),
    #('dim', LinearDiscriminantAnalysis(n_components=2)),
    #('clf', SVC(kernel='linear'))  # classifier
    ('clf', eclf_w)  # classifier
])


p_predictor = pipeline1.fit(train_set['Text'], 
                                  train_set.Score)

predicted = p_predictor.predict(dev_set['Text'].values)
print("Accuracy: ",accuracy_score(dev_set.Score, predicted))
print("Cohen's Kappa: ",cohen_kappa_score(dev_set.Score, predicted))

Accuracy:  0.636363636364
Cohen's Kappa:  0.386046511628


In [125]:
pd.crosstab(dev_set.Score, predicted, 
           rownames=['True'], colnames=['Predicted'], 
            margins=True)

Predicted,1,2,3,All
True,,,,
1,10,2,0,12
2,9,9,0,18
3,1,0,2,3
All,20,11,2,33


In [126]:
# Cross validation for checking final classifier

from sklearn.ensemble import VotingClassifier
from sklearn.cross_validation import KFold

# Get sizes of training and test sets
rows, columns = df.shape

kf = KFold(rows, 10, shuffle=True)
avg_accuracy = 0
count = 1
for train_indices, test_indices in kf:
    train_set = df.loc[train_indices] 
    dev_set = df.loc[test_indices] 
    
    pipeline1 = pipeline1.fit(train_set['Text'], 
                                  train_set.Score)
    
    trial_predictions = pipeline1.predict(dev_set["Text"])
    accuracy = accuracy_score(dev_set.Score, trial_predictions)
    print(count,": ",accuracy)
    avg_accuracy += accuracy_score(dev_set.Score, trial_predictions)
    count = count + 1
print("Average:",(avg_accuracy/10))

1 :  0.727272727273
2 :  0.787878787879
3 :  0.818181818182
4 :  0.666666666667
5 :  0.757575757576
6 :  0.727272727273
7 :  0.727272727273
8 :  0.575757575758
9 :  0.69696969697
10 :  0.636363636364
Average: 0.712121212121


In [176]:
# Need to examine some of the incorrectly classified responses
predicted_values = list(predicted)
actual = dev_set.values.tolist()

for (z,y) in zip(actual, predicted_values):
    if (str(z[1]) == str(y)):
        continue
    else:
        print("{}".format(z[0]))
        print("Graded as: {}".format(str(y)))
        print("Actual grade is {}".format(str(z[1])))
        print()
        #print(z)

We can put the thermometer in the sun instead of putting it in the shade. It will make it hotter if the heat goes to the thermometer.
Graded as: 3
Actual grade is 1

Because of our lack of organization of materials on the bottom of our oven,

The other group said we could improve by: evening out the placements of our tinfoil, and most importantly add a large amount of black paper inside of the oven attract heat inwards as well as outwards.
Graded as: 1
Actual grade is 3

1.) We think that if we add tin foil to the far side of the box inside wall and it will change the way radiation is radiated into the box and the black paper absorbing the heat energy coming of the tin foil.
2.) We are adding in more tape so that our box is more secure and the items we have on our box helping to heat up the box are going to stay still and work.
Graded as: 1
Actual grade is 2

Because... Then the radiation coming off the heat lamp is going to be better supervised and the heat energy will stay in the box